In [64]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm

In [65]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [66]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, results

In [67]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [68]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, lh, rh])


In [69]:
DATA_PATH = '/home/smayan/Desktop/ASL/dataset/SL'
sequence_length = 30
min_sequences_per_class = 10

In [70]:
actions = [
    'a', 'about', 'again', 'all', 'also', 'always', 'and', 'angry', 'animal', 'answer', 
    'apple', 'ask', 'baby', 'bad', 'bathroom', 'beautiful', 'because', 'bed', 'before', 
    'big', 'book', 'boy', 'brother', 'but', 'buy', 'bye', 'call', 'can', 'car', 'cat', 
    'city', 'class', 'clean', 'clothes', 'cold', 'college', 'color', 'come', 'computer', 
    'cook', 'dad', 'day', 'deaf', 'different', 'doctor', 'dog', 'done', "don't want", 
    'down', 'drink', 'eat', 'eight', 'enough', 'family', 'fast', 'father', 'feel', 
    'find', 'fine', 'finish', 'first', 'five', 'food', 'for', 'four', 'friend', 'from', 
    'get', 'girl', 'give', 'go', 'good', 'goodbye', 'happy', 'hard', 'have', 
    'head', 'hearing', 'hello', 'help', 'her', 'here', 'home', 'hospital', 'hot', 
    'house', 'how', 'hungry', 'i', 'if', 'in', 'know', 'language', 'last', 'later', 
    'learn', 'letter', 'like', 'little bit', 'live', 'look at', 'love', 'make', 'man', 
    'many', 'me', 'meet', 'milk', 'mom', 'money', 'month', 'more', 'morning', 'mother', 
    'movie', 'music', 'my', 'name', 'need', 'never', 'new', 'nice', 'night', 'nine', 
    'no', 'not', 'now', 'old', 'on', 'one', 'open', 'orange', 'our', 'out', 'people', 
    'phone', 'play', 'please', 'put', 'question', 'read', 'ready', 'red', 'right', 'sad', 
    'same', 'say', 'school', 'see', 'seven', 'she', 'shirt', 'shoes', 'show', 'sick', 
    'sign', 'sign language', 'sister', 'sit', 'six', 'sleep', 'slow', 'small', 'sorry', 
    'stand', 'start', 'stop', 'store', 'story', 'student', 'study', 'talk', 'teach', 
    'teacher', 'tell', 'ten', 'thank you', 'that']
# 'the', 'their', 'they', 'thing', 
#     'think', 'thirsty', 'this', 'three', 'time', 'tired', 'to', 'today', 'tomorrow', 
#     'two', 'understand', 'up', 'use', 'wait', 'walk', 'want', 'water', 'way', 
#     'we', 'wear', 'week', 'what', 'when', 'where', 'which', 'white', 'who', 'why', 
#     'will', 'with', 'woman', 'word', 'work', 'world', 'write', 'wrong', 'year', 'yellow', 
#     'yes', 'yesterday', 'you', 'your'
# ]
label_map = {label: num for num, label in enumerate(actions)}

In [71]:
len(actions)

178

In [72]:
sequences, labels = [], []

In [73]:
X = np.load('/media/smayan/500GB SSD/X.npy')
y = np.load('/media/smayan/500GB SSD/y.npy')

In [74]:
num_features = X.shape[2]
X = X.reshape(X.shape[0], X.shape[1], num_features, 1)

y_categorical = to_categorical(y, num_classes=len(actions))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)

In [75]:
X_train = X_train.squeeze(-1)  # Now shape = (6314, 30, 1530)
X_test  = X_test.squeeze(-1)

In [76]:
X.shape

(7893, 30, 1530, 1)

In [77]:
X_train.shape

(6314, 30, 1530)

In [78]:
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class weights computed: {class_weight_dict}")

Class weights computed: {0: 1.7054883318928262, 1: 1.0312255030049646, 2: 0.8868539325842697, 3: 0.9639716658524671, 4: 1.007788559754852, 5: 0.9238061797752809, 6: 1.4780898876404494, 7: 1.0557784911717496, 8: 0.7779420461265524, 9: 1.0312255030049646, 10: 0.6821953327571305, 11: 1.007788559754852, 12: 0.8062308478038815, 13: 0.8868539325842697, 14: 1.0815291860783776, 15: 1.1369922212618842, 16: 0.9434616304087975, 17: 0.6821953327571305, 18: 0.5992256301245066, 19: 0.6074342004001847, 20: 1.4780898876404494, 21: 1.0557784911717496, 22: 0.715204784342153, 23: 1.4780898876404494, 24: 0.9049529924329283, 25: 1.4780898876404494, 26: 0.726929452937926, 27: 0.8366546533813864, 28: 1.0557784911717496, 29: 0.8062308478038815, 30: 0.7918338683788122, 31: 0.7918338683788122, 32: 0.9049529924329283, 33: 1.4780898876404494, 34: 0.6618312929733355, 35: 0.9853932584269663, 36: 0.9049529924329283, 37: 1.2317415730337078, 38: 0.5156127515024823, 39: 0.9434616304087975, 40: 1.2317415730337078, 41: 0

In [79]:
model = Sequential()

model.add(Conv1D(256, kernel_size=3, activation='relu', input_shape=(sequence_length, 1530)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(actions), activation='softmax'))

/home/smayan/Desktop/AI-ML-DS/AI-and-ML-Course/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [80]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f'logs/wsl_model_{timestamp}'

In [81]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [82]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 28, 256)        │     1,175,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 28, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_12                │ (None, 14, 512)        │     1,050,624 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 14, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, 256)            │       656,384 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 178)            │        45,746 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,994,866 (11.42 MB)

 Trainable params: 2,994,354 (11.42 MB)

 Non-trainable params: 512 (2.00 KB)

In [83]:
X_train.shape

(6314, 30, 1530)

In [84]:
callbacks = [
    TensorBoard(log_dir=log_dir, histogram_freq=1),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-7)
]

In [86]:
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9310 - loss: 0.2053 - val_accuracy: 0.6852 - val_loss: 1.3176 - learning_rate: 2.0000e-04
Epoch 2/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9538 - loss: 0.1558 - val_accuracy: 0.6529 - val_loss: 1.4719 - learning_rate: 2.0000e-04
Epoch 3/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9536 - loss: 0.1544 - val_accuracy: 0.6935 - val_loss: 1.2754 - learning_rate: 2.0000e-04
Epoch 4/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9532 - loss: 0.1517 - val_accuracy: 0.7283 - val_loss: 1.1537 - learning_rate: 2.0000e-04
Epoch 5/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9515 - loss: 0.1604 - val_accuracy: 0.7492 - val_loss: 1.0826 - learning_rate: 2.0000e-04
Epoch 6/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9482 - loss: 0.1616 - val_accuracy: 0.7118 - val_loss: 1.2521 - learning_rate: 2.0000e-04
Epoch 7/150
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 

In [87]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


Test Loss: 0.9070
Test Accuracy: 79.23%


In [89]:
model.save(f'main_wsl_model_{timestamp}.h5')
print(f"\nModel saved as wsl_model_{timestamp}.h5")


Model saved as wsl_model_20250819-203608.h5
